# Transforming ROIs from MNI to the participant space

written by K. Garner, 2020
This code tranforms the pre-defined ROI masks into the participant T1w space for STRIAVISE WP1
Contact: getkellygarner@gmail.com

In [1]:
from bids.layout import BIDSLayout
from nipype import IdentityInterface
from nipype.interfaces import afni
from nipype.interfaces.ants import ApplyTransforms
from nipype.interfaces.io import BIDSDataGrabber, DataFinder, DataSink, DataGrabber
from nipype.pipeline import Node, MapNode, Workflow 
from nipype.interfaces.utility import Function
import pandas as pd
import os, re, json, glob

200518-11:11:13,597 nipype.utils INFO:
	 No new version available.


Set variables that apply across the workflow:

In [2]:
Basedir = "/scratch/qbi/uqkgarn1/STRIWP1/"
layout = BIDSLayout(Basedir)
tf_wf = Workflow(name="roi_tfs") # workflow to run the analysis
#rfl_wf = Workflow(name="rfl_roi_tfs")
subs = layout.get_subjects() # BIDSLayout does not appear to detect anything below the session folders, despite them being 
# BIDS compliant. However, the BIDSDataGrabber does appear to find the files, so pursuing for now
sess = layout.get_sessions()


Get a list of file names for each atlas

In [3]:
Keuken = ''.join([Basedir, 'derivatives/roi-masks/Keuken/*.nii.gz'])
KeukenAll = glob.glob(Keuken)

RFL = ''.join([Basedir, 'derivatives/roi-masks/RFL/*.nii.gz'])
RFLAll = glob.glob(RFL)
Atlases = [RFLAll, KeukenAll]

# define the one to be currently used in the workflow
atlas = 'Cort'

## User defined functions

In [4]:
def printSubPath(fullFilePath):
    # function to split filepath into constituent parts, then print string to add as input to DataSink for the container string
    # given the full filepath, this extracts the subject folder and session folder strings for input
    # into DataSink
    import os
    import re
    fname = os.path.normpath(fullFilePath)
    l = fname.split(os.sep)
    name = [[s for s in l if re.search('sub', s)][0]]
    name = '/'.join(name)
    return name

Now, define the nodes that form the workflow

# Data Grabber Nodes

First, a Data Grabber Node to grab the T1 data and the transform data

In [5]:
dgT1w = Node(DataGrabber(infields=['sub'], 
                         outfields=['anat','transform']), name='T1-grabber')
dgT1w.inputs.base_dir = "/scratch/qbi/uqkgarn1/STRIWP1/"
dgT1w.inputs.sort_filelist = True
dgT1w.inputs.template='*'
dgT1w.inputs.template_args = {'anat': [['sub', 'sub']],
                              'transform': [['sub']]}
dgT1w.inputs.field_template = {'anat': '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/sub-%s/anat/sub-%s_desc-preproc_T1w.nii.gz',
                               'transform': '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/sub-%s/anat/*from-MNI152NLin2009cAsym_to-T1w_mode-image_xfm.h5'}

First I test the node by directly entering a sub number, then I set up an IdentityInterface node to wrap around subjects.

In [ ]:
# dgT1w.inputs.sub = '01'
# dgT1w.run().outputs.transform

In [6]:
T1inf = Node(IdentityInterface(fields=['sub']),
             name='T1-info')
T1inf.iterables = [('sub', layout.get_subjects())]

Now use data grabber to get the ROI masks:

In [7]:
dgmk = Node(DataGrabber(), name='mask-grabber')
dgmk.inputs.base_directory = "/scratch/qbi/uqkgarn1/STRIWP1/derivatives/roi-masks/"
dgmk.inputs.template = [atlas + '/*.nii.gz'][0]
dgmk.inputs.sort_filelist = True
# res = dgmk.run()
# res.outputs

In [8]:
dgmk.inputs


base_directory = /scratch/qbi/uqkgarn1/STRIWP1/derivatives/roi-masks
drop_blank_outputs = False
field_template = <undefined>
raise_on_empty = True
sort_filelist = True
template = Cort/*.nii.gz
template_args = {'outfiles': []}

# Datasink Node

In [9]:
ds = Node(DataSink(), name='sink-stuff')
ds.inputs.base_directory = "/scratch/qbi/uqkgarn1/STRIWP1/derivatives/roi-masks/"
ds.inputs.container = atlas

In [10]:
substitutions = [('_sub_([0-9]*)', ''),
                 ('_transform([0-9]*)', '')
                ]


In [11]:
ds.inputs.regexp_substitutions = substitutions
ds.inputs


_outputs = {}
base_directory = /scratch/qbi/uqkgarn1/STRIWP1/derivatives/roi-masks/
bucket = <undefined>
container = Cort
creds_path = <undefined>
encrypt_bucket_keys = <undefined>
local_copy = <undefined>
parameterization = True
regexp_substitutions = [('_sub_([0-9]*)', ''), ('_transform([0-9]*)', '')]
remove_dest_dir = False
strip_dir = <undefined>
substitutions = <undefined>

# ANTS transform node


In [12]:
tf = MapNode(ApplyTransforms(), iterfield = ['input_image'], name='transform')
tf.inputs.interpolation='NearestNeighbor' # check fmri prep

In [13]:
tf.outputs

Bunch(output_image=<undefined>)

## options for ApplyTransforms

input_image - will be a list of the relevant mask files
interpolation - do I keep to Linear or change to NearestNeighbour (check Alex's)
reference_image = subjects T1w, fed through by the BIDS Data Grabber and a print name function
transforms = should grab these using the BIDS DataGrabber - need to make json files
dimension - can make into '3' if Ants can't detect automatically
output_image - a string of the output filename - can adapt one of the predefined functions to wrangle this 



## Connect workflow

In [14]:
tf_wf.connect([(T1inf, dgT1w, [('sub', 'sub')]),
               (dgT1w, ds,    [(('transform', printSubPath),
                                'container')]),
               (dgT1w, tf,    [('anat', 'reference_image')]),
               (dgT1w, tf,    [('transform', 'transforms')]),
               (dgmk,  tf,    [('outfiles', 'input_image')]),
               (tf,    ds,    [('output_image', atlas)])                          
              ])
tf_wf.run()

200518-11:11:39,462 nipype.workflow INFO:
	 Workflow roi_tfs settings: ['check', 'execution', 'logging', 'monitoring']
200518-11:11:39,488 nipype.workflow INFO:
	 Running serially.
200518-11:11:39,489 nipype.workflow INFO:
	 [Node] Setting-up "roi_tfs.T1-grabber" in "/tmp/tmplbrii2d_/roi_tfs/_sub_05/T1-grabber".
200518-11:11:39,495 nipype.workflow INFO:
	 [Node] Running "T1-grabber" ("nipype.interfaces.io.DataGrabber")
200518-11:11:39,502 nipype.workflow INFO:
	 [Node] Finished "roi_tfs.T1-grabber".
200518-11:11:39,504 nipype.workflow INFO:
	 [Node] Setting-up "roi_tfs.T1-grabber" in "/tmp/tmp_8w6czvq/roi_tfs/_sub_04/T1-grabber".
200518-11:11:39,509 nipype.workflow INFO:
	 [Node] Running "T1-grabber" ("nipype.interfaces.io.DataGrabber")
200518-11:11:39,514 nipype.workflow INFO:
	 [Node] Finished "roi_tfs.T1-grabber".
200518-11:11:39,515 nipype.workflow INFO:
	 [Node] Setting-up "roi_tfs.T1-grabber" in "/tmp/tmpvu0fjh_5/roi_tfs/_sub_03/T1-grabber".
200518-11:11:39,519 nipype.workflow IN

200518-11:12:07,783 nipype.workflow INFO:
	 [Node] Finished "roi_tfs.transform".
200518-11:12:07,784 nipype.workflow INFO:
	 [Node] Setting-up "roi_tfs.sink-stuff" in "/tmp/tmpmvpowwq_/roi_tfs/_sub_04/sink-stuff".
200518-11:12:07,792 nipype.workflow INFO:
	 [Node] Running "sink-stuff" ("nipype.interfaces.io.DataSink")
200518-11:12:07,794 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/roi-masks/sub-04/Cort/_sub_04/_transform0/FEF_trans.nii.gz -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/roi-masks/sub-04/Cort///FEF_trans.nii.gz
200518-11:12:07,798 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/roi-masks/sub-04/Cort/_sub_04/_transform1/IPS_trans.nii.gz -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/roi-masks/sub-04/Cort///IPS_trans.nii.gz
200518-11:12:07,801 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/roi-masks/sub-04/Cort/_sub_04/_transform2/LOC_trans.nii.gz -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/roi-masks/

200518-11:12:36,284 nipype.workflow INFO:
	 [Node] Finished "roi_tfs.sink-stuff".
200518-11:12:36,285 nipype.workflow INFO:
	 [Node] Setting-up "roi_tfs.transform" in "/tmp/tmp46_n9e5z/roi_tfs/_sub_01/transform".
200518-11:12:36,293 nipype.workflow INFO:
	 [Node] Setting-up "_transform0" in "/tmp/tmp46_n9e5z/roi_tfs/_sub_01/transform/mapflow/_transform0".
200518-11:12:36,298 nipype.workflow INFO:
	 [Node] Running "_transform0" ("nipype.interfaces.ants.resampling.ApplyTransforms"), a CommandLine Interface with command:
antsApplyTransforms --default-value 0 --float 0 --input /scratch/qbi/uqkgarn1/STRIWP1/derivatives/roi-masks/Cort/FEF.nii.gz --interpolation NearestNeighbor --output FEF_trans.nii.gz --reference-image /scratch/qbi/uqkgarn1/STRIWP1/derivatives/sub-01/anat/sub-01_desc-preproc_T1w.nii.gz --transform /scratch/qbi/uqkgarn1/STRIWP1/derivatives/sub-01/anat/sub-01_from-MNI152NLin2009cAsym_to-T1w_mode-image_xfm.h5
200518-11:12:41,12 nipype.workflow INFO:
	 [Node] Finished "_transfo